# Web Scrapping

Endereço Web : https://www.vivareal.com.br/venda/?pagina=1

capturar os seguintes campos em uma primeira rodada.

- Nome do imóvel
- Endereço
- Valor
- Descrição
- Quantidade de quartos
- Quantidade de banheiros
- Quantidade de vagas
- Area em m2
- Código do imóvel
- Telefone vendedor

Depois desta primeira ação paginar por um determinado número de páginas (dado pelo usuário)

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
from time import sleep
from random import randint

# Determinando a URL

In [2]:
url = "https://www.vivareal.com.br/venda/?pagina=1"

In [3]:
get(url)

<Response [200]>

### Capturando a Url com GET

In [5]:
vr = get(url).content

### Transformando o conteúdo em HTML

In [6]:
conteudo = BeautifulSoup(vr, 'html.parser')

In [7]:
conteudo

<!DOCTYPE html>

<html lang="pt-BR">
<head>
<script>
      (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
          (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
        m=s.getElementsByTagName(o)[0];a.src=g;m.parentNode.insertBefore(a,m)
      })(window,document,'script','https://www.google-analytics.com/analytics.js','ga');
      ga('create', 'UA-126375-31', 'auto');
      ga('require', 'GTM-PVK4X33');
    </script>
<script>
      var pageCategory = 'RESULT PAGE';
    
      const clickstreamTracker = function (rawData) {
        const clickstreamTrackerEndpoint = 'https://tracking.vivareal.com/events/v2'
        const data = JSON.stringify(rawData)
    
        if (navigator.sendBeacon) {
          navigator.sendBeacon(clickstreamTrackerEndpoint, data)
        } else {
          const ajax = new XMLHttpRequest()
          ajax.open('POST', clickstreamTrackerEndpoint, true)
          ajax.send(data)
        }
      };
  

# Identificando a posição inicial da relação de imóveis 

In [9]:
vivo_containers = conteudo.find_all('div', class_="js-card-selector")

### Verificando quantos itens foram coletados

In [10]:
print(len(vivo_containers))

36


# Testando as posições dos itens a serem coletados

In [11]:
first_vivo = vivo_containers[0]

In [53]:
first_vivo.h2.a.text.strip() #título do imóvel

'Apartamento com 3 Quartos à Venda, 70m²'

In [16]:
first_vivo.h2.find('span', class_='property-card__address').text.strip() #endereço do imóvel

'Rua Alfredo Pujol, 408 - Santana, São Paulo - SP'

In [20]:
first_vivo.find('div', class_='property-card__price').text.strip().replace("R$ ","").replace(".","") #valor do imóvel

'430000'

In [21]:
first_vivo.find('div', class_='property-card__description').text.strip() #descrição do imóvel

'O Grupo ZAP, marca que você conhece e confia, traz pra você uma excelente oportunidade: Morar no principal bairro da Zona Norte! Localizado em Santana, a 700 metros da linha azul do Metrô, o imóvel conta com 70 m², 3 dormitórios sendo 1 suíte e 2 banheiros.Condomínio com salão de festas, salão de jogos e playground.Próximo a restaurantes, academias e hospitais, a região oferece toda infraestrutura para você que procura por qualidade de vida, conforto e segurança!Além de um ótimo imóvel, aqui você encontra vendedores de confiança, imóvel verificado e financiamento facilitado.'

In [22]:
first_vivo.find('span', class_='property-card__detail-value').text.strip() #metros quadrados

'70'

In [23]:
first_vivo.find('span', class_='property-card__detail-value js-property-card-value').text.strip() #quantidade de quartos

'3'

In [44]:
first_vivo.find('li', class_='property-card__detail-item-extra').text.strip()[0:2].replace("\n","")#quantidade de suítes

'1'

In [43]:
first_vivo.find('li', class_='js-property-detail-bathroom').text.strip()[0:2].replace("\n","") #quantidade de banheiros

'2'

In [47]:
first_vivo.find('li', class_='js-property-detail-garages').text.strip()[0:2].replace("\n","") #quantidade de vagas

'1'

In [48]:
first_vivo.parent.get('id')

'2444209791'

In [49]:
first_vivo.find("a",class_="property-card__contact--phone").get('href').replace("tel:","+55")

'+551149351141'

# Função para coletar os dados com paginação

In [50]:
paginas = [str(i) for i in range(1,6)]
paginas

['1', '2', '3', '4', '5']

In [55]:
titulo = []
endereco = []
valor = []
descricao = []
metragem = []
quartos = []
suites = []
banheiros = []
vagas = []
id_imovel = []
telefone = []

for page in paginas:
    
    response = get('https://www.vivareal.com.br/venda/?pagina=' + page )    
    sleep(randint(10,30))     
    vivo_html = BeautifulSoup(response.content, 'html.parser')
    vivo_container = vivo_html.find_all('div', class_="js-card-selector")

    for container in vivo_container:
        
        tit = container.h2.a.text.strip()
        titulo.append(tit)

        end = container.h2.find('span', class_='property-card__address').text.strip()
        endereco.append(end)

        val = container.find('div', class_='property-card__price').text.strip().replace("R$ ","").replace(".","")
        valor.append(val)

        des = container.find('div', class_='property-card__description').text.strip()
        descricao.append(des)

        met = container.find('span', class_='property-card__detail-value').text.strip()
        metragem.append(met)

        qua = container.find('span', class_='property-card__detail-value js-property-card-value').text.strip()
        quartos.append(qua)

        sui = container.find('li', class_='property-card__detail-item-extra').text.strip()[0:2].replace("\n","")
        suites.append(sui)

        ban = container.find('li', class_='js-property-detail-bathroom').text.strip()[0:2].replace("\n","")
        banheiros.append(ban)

        vag = container.find('li', class_='js-property-detail-garages').text.strip()[0:2].replace("\n","")
        vagas.append(vag)

        idim = container.parent.get('id')
        id_imovel.append(idim)

        tel = container.find("a",class_="property-card__contact--phone").get('href').replace("tel:","+55")
        telefone.append(tel)
    
    

# Criando o DataFrame Paginado

In [56]:
df_paginado = pd.DataFrame({'titulo':titulo,
                            'endereco':endereco,
                            'valor':valor,
                            'descricao':descricao,
                            'metragem':metragem,
                            'quartos':quartos,
                            'suites':suites,
                            'banheiros':banheiros,
                            'vagas':vagas,
                            'id_imovel':id_imovel,
                            'telefone':telefone
                             })

In [57]:
df_paginado.head()

,titulo,endereco,valor,descricao,metragem,quartos,suites,banheiros,vagas,id_imovel,telefone
0,"Apartamento com 3 Quartos à Venda, 70m²","Rua Alfredo Pujol, 408 - Santana, São Paulo - SP",430000,"O Grupo ZAP, marca que você conhece e confia, ...",70,3,1,2,1,2444209791,+551149351141
1,"Apartamento com 3 Quartos à Venda, 129m²","Rua Dr. Paulo Vieira, 222 - Sumaré, São Paulo ...",1550000,Um apartamento 'Garden' da construtora Paulo M...,129,3,3,5,3,2447730721,+551149351141
2,"Apartamento com 2 Quartos à Venda, 66m²","Rua São Jorge, 125 - Parque São Jorge, São Pau...",499000,Conforto e comodidade a 700 m do metrô no Cond...,66,2,1,2,2,2448307885,+551149351141
3,"Apartamento com 2 Quartos à Venda, 101m²","Rua Cardoso de Almeida, 612 - Perdizes, São Pa...",638000,Apartamento no bairro Perdizes com 101 metros ...,101,2,1,2,1,2448891038,+551149351141
4,"Apartamento com 3 Quartos à Venda, 96m²","Rua Santo Egídio, 621 - Chora Menino, São Paul...",320000,Apartamento no bairro Chora Menino com 96 metr...,96,3,1,2,1,2448875188,+551149351141


In [58]:
df_paginado.size

1980